In [40]:
from datetime import date, timedelta, datetime
from typing import Generator, List
from itertools import islice

def generate_dates_from_today() -> Generator[date, None, None]:
    """
    A generator that yields dates starting from today, continuing indefinitely.

    Yields:
        date: The next date starting from today.
    """
    current_date = date.today()
    while True:
        yield current_date
        current_date += timedelta(days=1)

In [41]:
def rule_day_in_week(dates: Generator[date, None, None], trigger, days: List[str]) -> Generator[date, None, None]:
    """
    Yields dates from the generator where the weekday matches one of the days specified in the days list.
    
    Parameters:
        dates (Generator[date, None, None]): A generator that yields dates.
        trigger (Job): The Job object (unused in this function but kept for consistency with the signature).
        days (List[str]): A list of day names, like ["Monday", "Tuesday"].
        
    Yields:
        date: Dates where the weekday matches a day in the days list.
    """
    # Map day names to weekday numbers
    day_to_num = {"Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3, "Friday": 4, "Saturday": 5, "Sunday": 6}
    
    # Convert day names to numbers
    day_nums = [day_to_num[day] for day in days if day in day_to_num]

    # Yield dates where the weekday is in day_nums
    for day in dates:
        if day.weekday() in day_nums:
            yield day

In [42]:
def rule_day_in_year(dates: Generator[date, None, None], trigger, gte: str, lte: str) -> Generator[date, None, None]:
    """
    Yields dates from the generator filtered by a range defined by gte and lte tuples.
    
    Parameters:
        dates (Generator[date, None, None]): A generator that yields dates.
        trigger (Job): The Job object (unused in this function but kept for consistency).
        gte (str): Tuple of month name and day number, representing the start of the range.
        lte (str): Tuple of month name and day number, representing the end of the range.
        
    Yields:
        date: Dates within the specified range.
    """

    gte_date = tuple(int(x) for x in gte.split("-"))
    lte_date = tuple(int(x) for x in lte.split("-"))
    inner_date = lte_date > gte_date
    
    for date in dates:
        date_tuple = (date.month, date.day)
        # print(date_tuple, lte_date, gte_date, date_tuple <= lte_date and date_tuple >= gte_date if inner_date else date_tuple >= gte_date or date_tuple <= lte_date)
        if inner_date:
            if date_tuple <= lte_date and date_tuple >= gte_date:
                yield date
        else:
            if date_tuple >= gte_date or date_tuple <= lte_date:
                yield date
            

In [52]:
from datetime import date, timedelta
from typing import Generator

def rule_days_since(dates: Generator[date, None, None], trigger, event: str, days: int):
    """
    Yields dates from the generator 'dates' that are 'days' after the 'event' date in 'trigger'.

    :param dates: Generator of dates.
    :param trigger: Object containing the event date.
    :param event: String attribute name in 'trigger' to find the event date.
    :param days: Number of days after the event date to start yielding dates.

    :raises TypeError: If the event date in 'trigger' is not a valid date.
    """
    event_date_attr = f"{event}_date"
    reference_date = getattr(trigger, event_date_attr, None)

    # Validate the reference date
    if not isinstance(reference_date, date):
        raise TypeError(f"{event_date_attr} in trigger is not a valid date.")

    # Calculate the earliest date to start yielding from
    earliest_date = reference_date + timedelta(days=days)

    # Yield dates that are on or after the earliest date
    for current_date in dates:
        if current_date >= earliest_date:
            yield current_date

In [53]:
from collections import namedtuple

FakeJob = namedtuple("Job", ["due_date", "closed_date"])

In [58]:
import calendar
from datetime import date
from IPython.display import HTML
from itertools import islice

def generate_html_calendar(year, red_dates):
    # Create a calendar instance
    cal = calendar.HTMLCalendar(calendar.SUNDAY)

    # Start of HTML document
    html_calendar = f"""
    <html>
    <head>
    <style>
        table, th, td {{
            border: 1px solid black;
            border-collapse: collapse;
        }}
        th, td {{
            padding: 5px;
            text-align: center;
        }}
        .red {{
            color: red;
        }}
    </style>
    </head>
    <body>
    <h1>Calendar for {year}</h1>
    """

    # Generate each month's calendar
    for month in range(1, 13):
        month_calendar = cal.formatmonth(year, month)
        
        # Highlight the specific dates
        for day in red_dates:
            if day.year == year and day.month == month:
                # Replace the day number with a span element to add the red class
                month_calendar = month_calendar.replace(f'>{day.day}<', f'><span class="red">{day.day}</span><')

        # Add the month's calendar to the HTML
        html_calendar += month_calendar

    # End of HTML document
    html_calendar += """
    </body>
    </html>
    """

    return HTML(html_calendar)


# generate_html_calendar(2023, list(islice(
#     rule_day_in_week(
#     rule_day_in_year(generate_dates_from_today(), None, gte="04-10", lte="01-10")
#     , None, ["Monday"])
#     , 10000)))

generate_html_calendar(2023, list(islice(
    rule_day_in_year(
    rule_day_in_week(
    rule_days_since(generate_dates_from_today(), FakeJob(due_date=datetime.now(), closed_date=datetime.now()), event="due", days=10)
    , None, ["Monday", "Tuesday"])
    , None, lte="12-04", gte="01-01")
    
    , 1000)))
